Importing Required Packages

In [4]:
from langchain.document_loaders import UnstructuredPDFLoader, WebBaseLoader, AsyncChromiumLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough


In [3]:
#Constant KEY Variable (Use env in production)
KEY = "";

In [5]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=KEY)

In [19]:
#Scraping and Loading Data
loader = WebBaseLoader(web_paths=["https://academiccatalog.umd.edu/graduate/courses/cmsc/", "https://academiccatalog.umd.edu/graduate/courses/math/", "https://academiccatalog.umd.edu/graduate/courses/enes/", "https://academiccatalog.umd.edu/graduate/courses/bioe/", "https://academiccatalog.umd.edu/graduate/courses/econ/", "https://academiccatalog.umd.edu/graduate/courses/bmgt/" ])
data = loader.load();
print(data)

[Document(page_content='\n\n\n\n\n\n\n\nCMSC - Computer Science < University of Maryland\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to Content\nAZ Index\nCatalog Home\n\n\n\n\n\n\n\n\n\nTop Menu\n\n\nMenu Toggle\n\n\n\nUniversity of Maryland\n\n\n\n\nSearch UMD\n\nSubmit\n\n\n\n\n\n\nFlag\n\n\n                Admissions\n             \n\n\nFlag\n\n\n                Admissions\n             \n\n\nUndergraduate Admissions\n\n\nUndergraduate Majors\n\n\nVisit\n\n\nRequirements\n\n\nDeadlines\n\n\nCost\n\n\n\n\nGraduate Admissions\n\n\nGraduate Programs\n\n\nAdmissions Process\n\n\nFunding\n\n\n\nOther Links\n\n\nColleges & Schools\n\n\nFinancial Aid\n\n\nContact us\n\n\n\n\n\n\n\n\nStar\n\n\n                News\n             \n\n\n\n\n\n                      Calendar\n                   \n\n\n                Events\n             \n\n\n \n \n \n\nToggle navigation\n\n\n\n\n\n\n\n2023-2024 Catalog\nGraduate\n\n\n\nUndergraduate\nGraduate\n\nCourses A-Z\n\nUndergraduate\nGraduate

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
print(all_splits)

[Document(page_content='CMSC - Computer Science < University of Maryland\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to Content\nAZ Index\nCatalog Home\n\n\n\n\n\n\n\n\n\nTop Menu\n\n\nMenu Toggle\n\n\n\nUniversity of Maryland\n\n\n\n\nSearch UMD\n\nSubmit\n\n\n\n\n\n\nFlag\n\n\n                Admissions\n             \n\n\nFlag\n\n\n                Admissions\n             \n\n\nUndergraduate Admissions\n\n\nUndergraduate Majors\n\n\nVisit\n\n\nRequirements\n\n\nDeadlines\n\n\nCost\n\n\n\n\nGraduate Admissions\n\n\nGraduate Programs\n\n\nAdmissions Process\n\n\nFunding\n\n\n\nOther Links', metadata={'source': 'https://academiccatalog.umd.edu/graduate/courses/cmsc/', 'title': 'CMSC - Computer Science < University of Maryland', 'language': 'en'}), Document(page_content='Colleges & Schools\n\n\nFinancial Aid\n\n\nContact us\n\n\n\n\n\n\n\n\nStar\n\n\n                News\n             \n\n\n\n\n\n                      Calendar\n                   \n\n\n                Events\n

In [21]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings(openai_api_key=KEY))
retriever = vectorstore.as_retriever()
print(retriever)

tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x00000172DCC442D0>


In [22]:
#This is the prompt we give to OpenAI everytime we ask it a question
rag_prompt = hub.pull("rlm/rag-prompt")
print(rag_prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [24]:
rag_chain = (
                {"context": retriever, "question": RunnablePassthrough()}
                | rag_prompt
                | llm
        )

In [26]:
Question = "What courses should I take if I want to learn about start ups?"
awns = rag_chain.invoke(Question)
print(awns)

content='You should consider taking the following courses to learn about start-ups: Topics in Entrepreneurship, ENES460 Fundamentals of Technology Start-Up Ventures, and ENES672 Launching Technology Startup Ventures. These courses cover various aspects of technology entrepreneurship, including leadership, marketing, team-building, management, and launching new companies.'
